In [77]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 1. 뉴스 기사에서 추출하기
# CNN에서, 2023 FIFA 여자 월드컵 경기 결과 중 스코어 추출하기
import urllib.request
import bs4
import pandas as pd
import re

url='https://edition.cnn.com/2023/07/24/football/new-zealand-switzerland-norway-womens-world-cup-2023-spt-intl/index.html'
html=urllib.request.urlopen(url)
bs_obj=bs4.BeautifulSoup(html, 'html.parser')
div=bs_obj.find('div',{'class':'article__content'}).get_text()

# print(div.split('\n'))

print(re.findall('[0-9]-[0-9]', div))

['1-0', '1-0', '1-0', '2-0', '4-0', '6-0', '1-0', '0-0', '2-1', '1-0', '1-0', '5-0', '3-0', '3-0', '2-0', '0-0', '1-0', '1-0', '0-0', '2-0']


In [78]:
# 2, Wine 추천
# csv파일 불러오기
# 위에서 부터 5천개 끊기
import pandas as pd
df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/NLP/winemag-data-130k-v2.csv')
df=df.head(5000)

In [79]:
# 결측치 확인
wine=df['description']
wine.isnull().sum()

0

In [82]:
#  빈도수 및 불용어 처리 확인하기 위해 모듈 불러오기
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

# 전처리 전, 후 단어 개수 비교
refine=TfidfVectorizer()
refine.fit_transform(wine).toarray()
print(len(refine.vocabulary_)) # 전처리 전 단어 개수

# 불용어 사전 활용 및 빈도수 3이하인 데이터 삭제
refine=TfidfVectorizer(stop_words='english',min_df=3)
refine_final=refine.fit_transform(wine)
refine_final2=refine.fit_transform(wine).toarray()
print(refine_final.shape)
print(len(refine.vocabulary_)) # 전처리 후 단어 개수

8433
(5000, 3392)
3392


In [ ]:
refine_final

<5000x3392 sparse matrix of type '<class 'numpy.float64'>'
	with 109297 stored elements in Compressed Sparse Row format>

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cos_sim=cosine_similarity(refine_final)
cos_sim

array([[1.        , 0.01272033, 0.01279775, ..., 0.00850713, 0.        ,
        0.03136576],
       [0.01272033, 1.        , 0.02308952, ..., 0.00668297, 0.01616887,
        0.        ],
       [0.01279775, 0.02308952, 1.        , ..., 0.00672364, 0.01469438,
        0.03976313],
       ...,
       [0.00850713, 0.00668297, 0.00672364, ..., 1.        , 0.0120116 ,
        0.01478213],
       [0.        , 0.01616887, 0.01469438, ..., 0.0120116 , 1.        ,
        0.0145738 ],
       [0.03136576, 0.        , 0.03976313, ..., 0.01478213, 0.0145738 ,
        1.        ]])

In [ ]:
cos_sim.shape

(5000, 5000)

In [ ]:
df['title']

0                       Nicosia 2013 Vulkà Bianco  (Etna)
1           Quinta dos Avidagos 2011 Avidagos Red (Douro)
2           Rainstorm 2013 Pinot Gris (Willamette Valley)
3       St. Julian 2013 Reserve Late Harvest Riesling ...
4       Sweet Cheeks 2012 Vintner's Reserve Wild Child...
                              ...                        
4995    Mud House 2007 Swan Sauvignon Blanc (Marlborough)
4996     Fattoria Alois 2006 Cunto Pallagrello (Campania)
4997                      Florio NV Fine Sweet  (Marsala)
4998    Vice Versa 2005 Le Petit Vice Cabernet Sauvign...
4999    Viña Mar de Casablanca 2008 Reserva Especial S...
Name: title, Length: 5000, dtype: object

In [ ]:
df.index

RangeIndex(start=0, stop=5000, step=1)

In [ ]:
df.head(3)

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm


In [ ]:
data=dict(zip(df['title'], df.index))
i=data['Rainstorm 2013 Pinot Gris (Willamette Valley)']
print(i)

2


In [ ]:
data

{'Nicosia 2013 Vulkà Bianco  (Etna)': 0,
 'Quinta dos Avidagos 2011 Avidagos Red (Douro)': 1,
 'Rainstorm 2013 Pinot Gris (Willamette Valley)': 2,
 'St. Julian 2013 Reserve Late Harvest Riesling (Lake Michigan Shore)': 3,
 "Sweet Cheeks 2012 Vintner's Reserve Wild Child Block Pinot Noir (Willamette Valley)": 4,
 'Tandem 2011 Ars In Vitro Tempranillo-Merlot (Navarra)': 5,
 'Terre di Giurfo 2013 Belsito Frappato (Vittoria)': 6,
 'Trimbach 2012 Gewurztraminer (Alsace)': 7,
 'Heinz Eifel 2013 Shine Gewürztraminer (Rheinhessen)': 8,
 'Jean-Baptiste Adam 2012 Les Natures Pinot Gris (Alsace)': 9,
 'Kirkland Signature 2011 Mountain Cuvée Cabernet Sauvignon (Napa Valley)': 10,
 'Leon Beyer 2012 Gewurztraminer (Alsace)': 11,
 'Louis M. Martini 2012 Cabernet Sauvignon (Alexander Valley)': 12,
 'Masseria Setteporte 2012 Rosso  (Etna)': 13,
 'Mirassou 2012 Chardonnay (Central Coast)': 14,
 'Richard Böcking 2013 Devon Riesling (Mosel)': 15,
 'Felix Lavaque 2010 Felix Malbec (Cafayate)': 16,
 'Gaucho

In [ ]:
# t에 와인 이름, cosine_sim에 cos 유사도
def recommend(t, cosine_sim=cos_sim):
    # 와인의 인덱스 추출
    idx=data[t]
    sim_scores=list(enumerate(cosine_sim[idx]))
    print(sim_scores)
    ss=sorted(sim_scores, key=lambda x:x[1], reverse=True)
    ss=ss[1:11]
    print(ss)
    res=[i[0] for i in ss]
    print(res)
    return df['title'].iloc[res]

In [ ]:
recommend('Rainstorm 2013 Pinot Gris (Willamette Valley)')

[(0, 0.01279774587327889), (1, 0.02308951535178211), (2, 1.0), (3, 0.12693631016741685), (4, 0.005641680868337048), (5, 0.057629726428824364), (6, 0.018114562257227428), (7, 0.030919337176388645), (8, 0.02774319735721622), (9, 0.06444582528882953), (10, 0.006227615313910964), (11, 0.0534162880483523), (12, 0.007684043718644737), (13, 0.0), (14, 0.09918226614179443), (15, 0.0), (16, 0.04851237131758508), (17, 0.0), (18, 0.07804783958304704), (19, 0.0), (20, 0.038415209133215056), (21, 0.03902966698129929), (22, 0.07156111636729577), (23, 0.031490462487992574), (24, 0.0159234183338315), (25, 0.0), (26, 0.045639954744300035), (27, 0.0), (28, 0.0), (29, 0.061961877983827435), (30, 0.010960953341925617), (31, 0.006950509525312066), (32, 0.014195782378587608), (33, 0.022336078807051372), (34, 0.0760616937259681), (35, 0.0), (36, 0.04254061348965439), (37, 0.024454078196746602), (38, 0.00942112602888297), (39, 0.016742168553490328), (40, 0.0), (41, 0.02541105029318038), (42, 0.043573325200865

2763    River Road 2015 Un-Oaked Chardonnay (Sonoma Co...
2377    Airfield Estates 2012 Unoaked Chardonnay (Yaki...
3766    Long Meadow Ranch 2013 Sauvignon Blanc (Napa V...
1020    Stanton Vineyard 2013 Pinot Gris (Willamette V...
142          Banyan 2007 Riesling (Santa Lucia Highlands)
916     Coelho 2015 Renovação Estate Vineyards Pinot G...
3345               Chanson Père et Fils 2012  Viré-Clessé
4326    Chehalem 2007 INOX Chardonnay (Willamette Valley)
4191    Tin Barn 2013 Hi Vista Vineyard Sauvignon Blan...
3899    Plaisance Ranch 2014 Barrel Fermented Chardonn...
Name: title, dtype: object